In [6]:
from commons import *
import random
import itertools

# open the file with data

In [2]:
TORCH_DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TORCH_DTYPE = torch.float32

# Finding a 3 way-cut

In [7]:
from itertools import chain, islice, combinations
def find3WayCut(graph, terminal_lst):
    # Compute the first cut between the first two terminals
    cut_value_AB, partition_AB = nx.minimum_cut(graph, terminal_lst[0], terminal_lst[1], flow_func=shortest_augmenting_path)

    # Generate subgraphs based on the first cut
    graph_1 = graph.subgraph(partition_AB[0]).copy()
    graph_2 = graph.subgraph(partition_AB[1]).copy()

    # Determine in which subgraph the third terminal is located
    if terminal_lst[2] in partition_AB[0]:
        # If the third terminal is in graph_1, compute the minimum cut in graph_1
        # between terminal_lst[0] (or terminal_lst[1], depending on presence) and terminal_lst[2]
        if terminal_lst[0] in graph_1:
            cut_value_BC, _ = nx.minimum_cut(graph_1, terminal_lst[0], terminal_lst[2], flow_func=shortest_augmenting_path)
        else:
            cut_value_BC, _ = nx.minimum_cut(graph_1, terminal_lst[1], terminal_lst[2], flow_func=shortest_augmenting_path)
    else:
        # If the third terminal is in graph_2, compute the minimum cut in graph_2
        # between terminal_lst[0] (or terminal_lst[1], depending on presence) and terminal_lst[2]
        if terminal_lst[0] in graph_2:
            cut_value_BC, _ = nx.minimum_cut(graph_2, terminal_lst[0], terminal_lst[2], flow_func=shortest_augmenting_path)
        else:
            cut_value_BC, _ = nx.minimum_cut(graph_2, terminal_lst[1], terminal_lst[2], flow_func=shortest_augmenting_path)

    # Sum the cut values to get the total 3-way cut value
    return cut_value_AB + cut_value_BC
def recursive_cut(graph, terminals):
    if len(terminals) <= 1 or graph.number_of_nodes() == 0:
        # Base case: no cut needed if only one terminal or graph is empty
        return 0, {terminals[0]: graph} if terminals else {}

    # Perform initial 2-way cut between the first two terminals
    cut_value, (part_1, part_2) = nx.minimum_cut(graph, terminals[0], terminals[1], flow_func=shortest_augmenting_path)
    graph_1, graph_2 = graph.subgraph(part_1).copy(), graph.subgraph(part_2).copy()

    # Determine which terminals are in each subgraph
    terminals_1, terminals_2 = [], []
    for terminal in terminals:
        if terminal in part_1:
            terminals_1.append(terminal)
        else:
            terminals_2.append(terminal)

    # Recursively apply cuts to each subgraph
    cut_value_1, partitions_1 = recursive_cut(graph_1, terminals_1)
    cut_value_2, partitions_2 = recursive_cut(graph_2, terminals_2)

    # Combine the results
    total_cut_value = cut_value + cut_value_1 + cut_value_2
    partitions = partitions_1
    partitions.update(partitions_2)  # Merge the two partition dictionaries

    return total_cut_value, partitions

def find_k_way_cut(graph, terminals):
    """
    Find a k-way cut in the graph such that all terminals are in separate partitions.

    Parameters:
    - graph: A NetworkX graph.
    - terminals: A list of terminal nodes that need to be in separate partitions.

    Returns:
    - The total cut value for separating all terminals into their own partition.
    - A dictionary mapping each terminal to its corresponding subgraph.
    """

    permutations = list(itertools.permutations(terminals))
    best_cut_Value = torch.inf
    best_partition = []
    for perm in permutations:
        total_cut_value, partitions = recursive_cut(graph, perm)
        if total_cut_value < best_cut_Value:
            best_cut_Value = total_cut_value
            best_partition = partitions

    # total_cut_value, partitions = recursive_cut(graph, terminals)
    # return total_cut_value, partitions
    return best_cut_Value, best_partition

In [8]:

def qubo_dict_to_torch(nx_G, Q, torch_dtype=None, torch_device=None):
    """
    Output Q matrix as torch tensor for given Q in dictionary format.

    Input:
        Q: QUBO matrix as defaultdict
        nx_G: graph as networkx object (needed for node lables can vary 0,1,... vs 1,2,... vs a,b,...)
    Output:
        Q: QUBO as torch tensor
    """

    # get number of nodes
    n_nodes = len(nx_G.nodes)

    # get QUBO Q as torch tensor
    Q_mat = torch.zeros(n_nodes, n_nodes)
    for (x_coord, y_coord), val in Q.items():
        Q_mat[x_coord][y_coord] = val

    if torch_dtype is not None:
        Q_mat = Q_mat.type(torch_dtype)

    if torch_device is not None:
        Q_mat = Q_mat.to(torch_device)

    return Q_mat
def gen_adj_matrix(nx_G):
    adj_dict = defaultdict(int)

    for(u,v) in nx_G.edges:
        adj_dict[(u, v)] = nx_G[u][v]['weight']
        adj_dict[(v, u)] = nx_G[u][v]['weight']

    for u in nx_G.nodes:
        for i in nx_G.nodes:
            if not adj_dict[(u, i)]:
                adj_dict[(u, i)] = 0

    return adj_dict

In [18]:
heurestic_cut_3 = []
test_item = open_file( filename='./testData/prepareDS.pkl')

# pos = nx.kamada_kawai_layout(test_item[1][2])
# nx.draw(test_item[1][2], pos, with_labels=True, node_color=[[.7, .7, .7]])
# labels = nx.get_edge_attributes(test_item[1][2],'weight')
# nx.draw_networkx_edge_labels(test_item[1][2],pos,edge_labels=labels)
## new - remove
# test_item_2 = {}
# test_item_2[0]=test_item[1]
# print(test_item_2[0][-1])
##
i = 0
for key, (dgl_graph, adjacency_matrix,graph, terminal) in test_item.items():

    l = find_k_way_cut(graph, terminal)
    heurestic_cut_3.append(l[0])
    print(i, ": number :", "Heurestic k-way 3 min-cut value:" + str(heurestic_cut_3[-1]), l[1][terminal[0]].number_of_nodes(), l[1][terminal[1]].number_of_nodes(), l[1][terminal[2]].number_of_nodes(), "; Terminals - "+ str(terminal))
    # print(find_k_way_cut(graph, [0,1,2]))
    #
    # swap_graph_nodes(graph, {57:0, 56:1, 47:2, 0:57, 1:56, 2:47})
    # # print("Edges of the new graph:", graph.edges())
    # # print("Edges of the new graph:", graph.edges())
    # print(find_k_way_cut(graph, terminal))
    # print(find_k_way_cut(graph, [0,1,2]))

    i+=1

    if i>10:
        break


    #
    # heurestic_cut_k.append(find_k_way_cut(graph, [0,20,60,85,98])[0])
    # print("Heurestic k-way 5 min-cut value: " + str(heurestic_cut_k[-1]))

0 : number : Heurestic k-way 3 min-cut value:92.0 15 61 4 ; Terminals - [0, 1, 2]
1 : number : Heurestic k-way 3 min-cut value:2.0 20 31 29 ; Terminals - [0, 1, 2]
2 : number : Heurestic k-way 3 min-cut value:20.0 56 5 19 ; Terminals - [0, 1, 2]
3 : number : Heurestic k-way 3 min-cut value:98.0 48 23 9 ; Terminals - [0, 1, 2]
4 : number : Heurestic k-way 3 min-cut value:6.0 40 29 11 ; Terminals - [0, 1, 2]
5 : number : Heurestic k-way 3 min-cut value:121.0 14 4 62 ; Terminals - [0, 1, 2]
6 : number : Heurestic k-way 3 min-cut value:32.0 43 28 9 ; Terminals - [0, 1, 2]
7 : number : Heurestic k-way 3 min-cut value:6.0 7 23 50 ; Terminals - [0, 1, 2]
8 : number : Heurestic k-way 3 min-cut value:41.0 50 24 6 ; Terminals - [0, 1, 2]
9 : number : Heurestic k-way 3 min-cut value:18.0 9 2 69 ; Terminals - [0, 1, 2]
10 : number : Heurestic k-way 3 min-cut value:29.0 22 53 5 ; Terminals - [0, 1, 2]


In [15]:
heurestic_cut_3 = []
test_item = open_file( filename='./testData/prepareDS.pkl')

# pos = nx.kamada_kawai_layout(test_item[1][2])
# nx.draw(test_item[1][2], pos, with_labels=True, node_color=[[.7, .7, .7]])
# labels = nx.get_edge_attributes(test_item[1][2],'weight')
# nx.draw_networkx_edge_labels(test_item[1][2],pos,edge_labels=labels)
## new - remove
# test_item_2 = {}
# test_item_2[0]=test_item[1]
# print(test_item_2[0][-1])
##
i = 0
for key, (dgl_graph, adjacency_matrix,graph, terminal) in test_item.items():

    l = find_k_way_cut(graph, [0,1])
    heurestic_cut_3.append(l[0])
    print("Heurestic k-way 2 min-cut value: " + str(heurestic_cut_3[-1]),":", l[1][terminal[0]].number_of_nodes(), l[1][terminal[1]].number_of_nodes(), "; Terminals - "+ str(terminal))
    # print(find_k_way_cut(graph, [0,1,2]))
    #
    # swap_graph_nodes(graph, {57:0, 56:1, 47:2, 0:57, 1:56, 2:47})
    # # print("Edges of the new graph:", graph.edges())
    # # print("Edges of the new graph:", graph.edges())
    # print(find_k_way_cut(graph, terminal))
    # print(find_k_way_cut(graph, [0,1,2]))

    i+=1

    if i>10:
        break


    #
    # heurestic_cut_k.append(find_k_way_cut(graph, [0,20,60,85,98])[0])
    # print("Heurestic k-way 5 min-cut value: " + str(heurestic_cut_k[-1]))

Heurestic k-way 2 min-cut value: 1.0 : 15 65 ; Terminals - [0, 1, 2]
Heurestic k-way 2 min-cut value: 1.0 : 49 31 ; Terminals - [0, 1, 2]
Heurestic k-way 2 min-cut value: 19.0 : 75 5 ; Terminals - [0, 1, 2]
Heurestic k-way 2 min-cut value: 8.0 : 48 32 ; Terminals - [0, 1, 2]
Heurestic k-way 2 min-cut value: 1.0 : 51 29 ; Terminals - [0, 1, 2]
Heurestic k-way 2 min-cut value: 40.0 : 14 66 ; Terminals - [0, 1, 2]
Heurestic k-way 2 min-cut value: 30.0 : 43 37 ; Terminals - [0, 1, 2]
Heurestic k-way 2 min-cut value: 3.0 : 57 23 ; Terminals - [0, 1, 2]
Heurestic k-way 2 min-cut value: 3.0 : 56 24 ; Terminals - [0, 1, 2]
Heurestic k-way 2 min-cut value: 17.0 : 78 2 ; Terminals - [0, 1, 2]
Heurestic k-way 2 min-cut value: 1.0 : 22 58 ; Terminals - [0, 1, 2]


In [204]:
nx.minimum_cut(test_item_2[0][2], test_item_2[0][3][1], test_item_2[0][3][0], flow_func=shortest_augmenting_path)

(1.0,
 ({0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   53,
   55,
   56,
   58,
   60,
   61,
   63,
   64,
   65,
   66,
   67,
   68,
   70,
   74,
   75,
   76},
  {52, 54, 57, 59, 62, 69, 71, 72, 73, 77, 78, 79}))

In [181]:
heurestic_cut_3 = []
heurestic_cut_k = []
test_item = open_file(obj = test_item, filename='./testData/400-600.pkl')

# pos = nx.kamada_kawai_layout(test_item[1][2])
# nx.draw(test_item[1][2], pos, with_labels=True, node_color=[[.7, .7, .7]])
# labels = nx.get_edge_attributes(test_item[1][2],'weight')
# nx.draw_networkx_edge_labels(test_item[1][2],pos,edge_labels=labels)

for key, (dgl_graph, adjacency_matrix,graph) in test_item.items():
    cut_value, (part_1, part_2) = nx.minimum_cut(graph, 200, 399)

    heurestic_cut_3.append(cut_value)
    print("Heurestic 2-way min-cut value: " + str(heurestic_cut_3[-1]), len(part_1), len(part_2))

    l = find_k_way_cut(graph, [0,200,399])
    # print(l)
    heurestic_cut_k.append(l[0])
    print("Heurestic k-way 3 min-cut value: " + str(heurestic_cut_k[-1]), l[1][0].number_of_nodes(), l[1][200].number_of_nodes(), l[1][399].number_of_nodes())

    l = find_k_way_cut(graph, [0,200,299, 399])
    # print(l)
    heurestic_cut_k.append(l[0])
    print("Heurestic k-way 3 min-cut value: " + str(heurestic_cut_k[-1]), l[1][0].number_of_nodes(), l[1][200].number_of_nodes(), l[1][399].number_of_nodes(), l[1][299].number_of_nodes())
    #
    # heurestic_cut_k.append(find_k_way_cut(graph, [0,20,60,85,98])[0])
    # print("Heurestic k-way 5 min-cut value: " + str(heurestic_cut_k[-1]))

Heurestic 2-way min-cut value: 27705 562 1


KeyboardInterrupt: 

In [89]:
def CreateDummyFunction2():
    test_graph = nx.Graph()
    test_graph.add_edges_from([(0,1, {"weight": 1, "capacity":5}),
                               (0,2, {"weight": 2, "capacity":2}),
                               (1,3, {"weight": 2, "capacity":2}),
                               (2,3, {"weight": 2, "capacity":2}),
                               (0,4, {"weight": 100, "capacity":100}),
                               (2,5, {"weight": 100, "capacity":100})])
    test_graph.order()
    return test_graph


In [29]:
heurestic_cut_3 = []
heurestic_cut_k = []

for key, (dgl_graph, adjacency_matrix,graph) in test_item.items():


    heurestic_cut_k.append(find_k_way_cut(graph, [0,20,60,85,98])[0])
    print("Heurestic k-way 5 min-cut value: " + str(heurestic_cut_k[-1]))

Heurestic k-way 5 min-cut value: 21140
Heurestic k-way 5 min-cut value: 27131
Heurestic k-way 5 min-cut value: 21814
Heurestic k-way 5 min-cut value: 23645
Heurestic k-way 5 min-cut value: 23056
Heurestic k-way 5 min-cut value: 33226
Heurestic k-way 5 min-cut value: 32765
Heurestic k-way 5 min-cut value: 28692
Heurestic k-way 5 min-cut value: 33983
Heurestic k-way 5 min-cut value: 23906


In [30]:
save_object(heurestic_cut_k, './testData/heurestic_cut_k.pkl')

In [16]:
# import cplex
# from cplex.exceptions import CplexError
#
# def setup_3way_cut_problem(G, terminals):
#     try:
#         # Initialize CPLEX problem
#         prob = cplex.Cplex()
#         prob.set_problem_type(cplex.Cplex.problem_type.LP)
#         prob.objective.set_sense(prob.objective.sense.minimize)
#
#         # Add variables
#         edge_vars = {}
#         node_partition_vars = {}
#         for i in G.nodes():
#             if i in terminals:
#                 continue  # Skip terminals
#             for t in range(3):  # For each terminal/partition
#                 var_name = f"x_{i}_{t}"
#                 node_partition_vars[(i, t)] = var_name
#                 prob.variables.add(names=[var_name], lb=[0], ub=[1], types=["B"])
#
#         for i, j in G.edges():
#             var_name = f"y_{i}_{j}"
#             edge_vars[(i, j)] = var_name
#             prob.variables.add(names=[var_name], lb=[0], ub=[1], types=["B"], obj=[G[i][j]['weight']])
#
#         # Add constraints
#         # Node must belong to one partition
#         for i in G.nodes():
#             if i in terminals:
#                 continue
#             constraints = [node_partition_vars[(i, t)] for t in range(3)]
#             prob.linear_constraints.add(
#                 lin_expr=[cplex.SparsePair(ind=constraints, val=[1]*3)],
#                 senses=["E"],
#                 rhs=[1]
#             )
#
#         # Edge cut constraints
#         for i, j in G.edges():
#             for t in range(3):
#                 prob.linear_constraints.add(
#                     lin_expr=[
#                         cplex.SparsePair(ind=[node_partition_vars[(i, t)], node_partition_vars[(j, t)], edge_vars[(i, j)]],
#                                          val=[1, -1, 1]),
#                         cplex.SparsePair(ind=[node_partition_vars[(i, t)], node_partition_vars[(j, t)], edge_vars[(i, j)]],
#                                          val=[-1, 1, 1])
#                     ],
#                     senses="LL",
#                     rhs=[0, 0]
#                 )
#
#         # Solve the problem
#         prob.solve()
#         return prob, node_partition_vars
#
#     except CplexError as exc:
#         print(exc)
#         return None
#
# def extract_partitions(problem, G, terminals, node_partition_vars):
#     # Retrieve the values of the decision variables
#     variable_values = problem.solution.get_values()
#
#     # Initialize dictionaries to hold the nodes for each terminal's partition
#     partitions = {terminal: [] for terminal in terminals}
#
#     # Variable names mapping back to node and terminal
#     var_to_node = {v: k for k, v in node_partition_vars.items()}
#
#     # Loop through all variable values and assign nodes to partitions based on variable value
#     for var_index, value in enumerate(variable_values):
#         var_name = problem.variables.get_names(var_index)
#         if var_name.startswith('x_') and value > 0.5:  # Considering binary variables true if > 0.5
#             node, terminal_index = var_to_node[var_name]
#             partitions[terminals[terminal_index]].append(node)
#
#     return partitions
#
# # Example usage
# G =  test_item[0][2] # Your graph initialization here
# terminals = ['0', '20', '98']  # Your terminals
# problem = setup_3way_cut_problem(G, terminals)
# if problem:
#     # Extract solution, etc.
#     print("Solution status = ", problem[0].solution.get_status(), ":", end=' ')
#     print(problem[0].solution.status[problem[0].solution.get_status()])
#     print("Solution value  = ", problem[0].solution.get_objective_value())
#
#     # Extract and print the partitions
#     partitions = extract_partitions(problem[0], G, terminals, problem[1])
#     for terminal, nodes in partitions.items():
#         print(f"Partition for terminal {terminal}: {nodes}")


KeyError: (0, 0)

In [21]:
import cplex
from cplex.exceptions import CplexError

def setup_3way_cut_problem(G, terminals):
    try:
        # Initialize CPLEX problem
        prob = cplex.Cplex()
        prob.objective.set_sense(prob.objective.sense.minimize)

        # Ensure variables for each node with respect to each terminal
        # Initialize dictionaries for variables
        node_vars = {}
        edge_vars = {}

        # Add variables for node-partition assignments
        for i in G.nodes():
            for t_index, t in enumerate(terminals):
                var_name = f"x_{i}_{t_index}"
                if str(i) == t:  # Force terminal nodes to their respective partitions
                    prob.variables.add(names=[var_name], lb=[1], ub=[1], types=["B"])
                else:
                    prob.variables.add(names=[var_name], lb=[0], ub=[1], types=["B"])
                node_vars[(i, t_index)] = var_name

        # Add variables for edges indicating if they are cut
        for i, j in G.edges():
            var_name = f"y_{i}_{j}"
            edge_vars[(i, j)] = var_name
            weight = G[i][j]['weight']
            prob.variables.add(names=[var_name], lb=[0], ub=[1], types=["B"], obj=[weight])

        # Constraint: Each non-terminal node must belong to exactly one partition
        for i in G.nodes():
            if str(i) not in terminals:  # Skip terminal assignment here
                prob.linear_constraints.add(
                    lin_expr=[[
                        [node_vars[(i, t_index)] for t_index in range(3)],  # Variables for this node across partitions
                        [1.0] * 3  # Coefficients
                    ]],
                    senses=["E"],  # Equality
                    rhs=[1.0]  # Must be in exactly one partition
                )

        # Constraint: For each edge, enforce cut if nodes are in different partitions
        for i, j in G.edges():
            for t_index in range(3):
                prob.linear_constraints.add(
                    lin_expr=[[
                        [node_vars[(i, t_index)], node_vars[(j, t_index)], edge_vars[(i, j)]],
                        [1, -1, -G.number_of_nodes()],  # Large negative coefficient to enforce y_ij = 1 if i and j differ in partition
                        [-1, 1, -G.number_of_nodes()]   # Same here
                    ]],
                    senses=["L", "L"],  # Less than or equal, to activate edge_var if nodes are in different partitions
                    rhs=[0, 0]
                )

        # Solve the problem
        prob.solve()
        return prob, node_vars

    except CplexError as exc:
        print(exc)
        return None

def extract_partitions(problem, G, terminals, node_vars):
    partitions = {t: [t] for t in terminals}  # Initialize partitions with terminals
    solution_values = problem.solution.get_values()

    for (i, t_index), var_name in node_vars.items():
        value = solution_values[problem.variables.get_indices(var_name)]
        if value > 0.5 and str(i) not in terminals:  # Node assigned to this terminal's partition
            partitions[terminals[t_index]].append(str(i))

    return partitions

# Assuming G has been created and terminals are defined
G =  test_item[0][2] # Your graph initialization here
terminals = ['0', '20', '98']  # Assuming these are the string representations of your terminal nodes
print(setup_3way_cut_problem(G, terminals))
problem, node_vars = setup_3way_cut_problem(G, terminals)
if problem:
    print("Solution status:", problem.solution.get_status(), ":", problem.solution.status[problem.solution.get_status()])
    print("Solution value:", problem.solution.get_objective_value())
    partitions = extract_partitions(problem, G, terminals, node_vars)
    for t, nodes in partitions.items():
        print(f"Partition for terminal {t}: {nodes}")


ValueError: too many values to unpack (expected 2)

In [23]:
import cplex
from cplex.exceptions import CplexError
import networkx as nx
import random

def setup_3way_cut_problem(G, terminals):
    try:
        # Initialize CPLEX problem
        prob = cplex.Cplex()
        prob.objective.set_sense(prob.objective.sense.minimize)

        node_vars = {}  # Node-partition assignment variables
        edge_vars = {}  # Edge-cut indicator variables

        # Add node variables
        for i in G.nodes():
            for t, terminal in enumerate(terminals):
                var_name = f"x_{i}_{t}"
                prob.variables.add(names=[var_name], lb=[0], ub=[1], types=["B"])
                node_vars[(i, t)] = var_name

        # Add edge variables and set the objective to minimize cut weights
        for i, j, data in G.edges(data=True):
            var_name = f"y_{i}_{j}"
            weight = data['weight']  # Extracting the weight from the edge data
            prob.variables.add(names=[var_name], lb=[0], ub=[1], types=["B"], obj=[weight])
            edge_vars[(i, j)] = var_name

        # Each node must belong to exactly one partition
        for i in G.nodes():
            constraints = [node_vars[(i, t)] for t in range(3)]
            prob.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind=constraints, val=[1.0]*3)],
                senses=["E"],
                rhs=[1.0]
            )

        # Enforcing edge cuts if nodes are in different partitions
        for (i, j), var_name in edge_vars.items():
            for t in range(3):
                node_i_var = node_vars[(i, t)]
                node_j_var = node_vars[(j, t)]
                # For each edge and each terminal, add constraints to enforce the cut
                prob.linear_constraints.add(
                    lin_expr=[
                        cplex.SparsePair(ind=[node_i_var, node_j_var, var_name], val=[1, -1, G.number_of_nodes()]),
                        cplex.SparsePair(ind=[node_i_var, node_j_var, var_name], val=[-1, 1, G.number_of_nodes()])
                    ],
                    senses=["L", "L"],
                    rhs=[G.number_of_nodes() - 1, G.number_of_nodes() - 1]
                )

        prob.solve()
        return prob, node_vars

    except CplexError as exc:
        print(exc)
        return None

def extract_partitions(problem, G, terminals, node_vars):
    partitions = {t: [] for t in terminals}
    solution_values = problem.solution.get_values()

    # Map variables back to their node and terminal
    for (node, terminal), var in node_vars.items():
        value = solution_values[problem.variables.get_indices(var)]
        if value > 0.5:  # Node is in this terminal's partition
            partitions[terminals[terminal]].append(node)

    return partitions

# Example of creating a graph and using the setup
def create_example_graph():
    G = nx.Graph()
    G.add_nodes_from(range(101))  # 101 nodes numbered from 0 to 100
    # Add random weighted edges
    for i in range(101):
        for j in range(i+1, 101):
            if random.random() < 0.5:  # Add an edge with 50% probability
                weight = random.randint(1, 10)
                G.add_edge(i, j, weight=weight)
    return G

# Initialize a graph
#G = create_example_graph()
G =  test_item[0][2]
# Define terminals (ensure they are within the graph's node range)
terminals = ['0', '20', '98']  # Example terminals as strings, adjust according to your needs

# Setup and solve the 3-way cut problem
problem, node_vars = setup_3way_cut_problem(G, terminals)
if problem:
    print("Solution status: ", problem.solution.get_status())
    print("Objective value: ", problem.solution.get_objective_value())
    partitions = extract_partitions(problem, G, terminals, node_vars)
    for terminal, nodes in partitions.items():
        print(f"Partition {terminal}: {nodes}")


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.03 sec. (1.59 ticks)

Root node processing (before b&c):
  Real time             =    0.04 sec. (1.77 ticks)
Parallel b&c, 10 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.04 sec. (1.77 ticks)
Solution status:  101
Objective value:  0.0
Partition 0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107]
Partition